[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana7/RegLin_vs_RedNeu.ipynb)

# Introducción

En este notebook se utilizará un dataset para regresión, con la finalidad de comparar el desempeño entre una regresión lineal y un par de redes neuronales. 

Se verá la importación de las depencias, la carga de los datos, el uso de [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) junto a [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) para conectar facilmente al procesamiento de los datos con el algoritmo (en este caso, regresión lineal). 

También, se verá la creación de las redes neuronales y el uso de callbacks (en particular, guardar checkpoints) durante el entrenamiento.

Para todos los modelos se evaluará el desempeño que se obtuvo y se harán comparaciones entre los tres.

# Dependencias

In [ ]:
# !pip install -U plotly

In [ ]:
import os

# Manejo de los datos
import pandas as pd
import numpy as np

# Utilidades, modelo y métricas
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Para la red neuronal
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# Visualización
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
pio.templates.default = "plotly_white"

In [ ]:
# directorio para guardar los checkpoints de las redes neuronales, para código local
# checkpoint_filepath = 'checkpoints'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Funciones

In [ ]:
def plot_model_evaluation(real_train, predecido_train, real_test, predecido_test, plot_title):
    
    fig = make_subplots(
        rows = 1, cols = 2, 
        subplot_titles = ['Conjunto de entrenamiento', 'Conjunto de prueba'],        
    )

    fig.add_trace(
        go.Scatter(
            x = real_train, 
            y = predecido_train, 
            mode='markers', 
            ), 
        row = 1, col = 1
    )

    fig.add_shape(
        type = 'line', 
        x0 = np.min(predecido_train), y0 = np.min(predecido_train), 
        x1 = np.max(predecido_train), y1 = np.max(predecido_train), 
        line=dict(dash = 'dot')
    )

    fig.add_trace(
        go.Scatter(x = real_test, y = predecido_test, mode = 'markers'), 
        row = 1, col = 2
    )

    fig.add_shape(
        type = 'line', 
        x0 = np.min(predecido_test), y0 = np.min(predecido_test), 
        x1 = np.max(predecido_test), y1 = np.max(predecido_test), 
        line=dict(dash = 'dot'), 
        row = 1, col = 2
    )
    
    fig.update_layout(
        showlegend=False, 
        title_text=plot_title
    )

    fig.update_xaxes(title_text = 'Cargos reales', row = 1, col = 1)
    fig.update_xaxes(title_text = 'Cargos reales', row = 1, col = 2)
    fig.update_yaxes(title_text = 'Cargos predichos', row = 1, col = 1)
    fig.update_yaxes(title_text = 'Cargos predichos', row = 1, col = 2)

    return fig


# Datos

Se trabajará con los datos de [Medical Cost Personal Datasets](https://www.kaggle.com/mirichoi0218/insurance). Contiene 7 columnas descritas a continuación:



* age: Edad del beneficiario del seguro

* sex: Género del contratante del seguro (female, male)

* bmi: Índice de masa corporal, se obtiene como el pesó entre la estatura del paciente (kg / m ^ 2) usando la escala convencional, idealmente de 18.5 a 24.9

* children: Número de niños que cubre el seguro (Dependientes del asegurado)

* smoker: Fumador

* region: Donde se encuentra el área residencial del beneficiario en EUA (northeast, southeast, southwest, northwest)

* charges: Cargos de gastos médicos cubiertos por el seguro


Se quiere saber si se pueden predecir los cargos (charges) en función de las demás variables.

## Carga de los datos

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [ ]:
data.isnull().sum().sum()

0

## Descripción de los datos

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


## División en conjuntos de entrenamiento y prueba

Separamos el DataFrame en dos: 
1. X: Contiene las variables regresoras.
2. y: La variable de respuesta.

In [ ]:
X = data.drop(columns = 'charges')
y = data['charges']

Ahora divimos el dataset en el conjunto de entrenamiento y de prueba. Esto, como hemos visto, nos ayuda a evaluar si el modelo "aprendió" a generalizar las predicciones o si se sobreajustó.

El conjunto de entrenamiento tendrá el 70% de los datos totales.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(936, 6) (936,) (402, 6) (402,)


# Regressión Lineal

Primero veremos los resultados que obtenemos con una regresión lineal (múltiple). Sin entrar en detalle a los supuestos del algoritmo, la hipótesis es la siguiente.

\begin{align}
    y_i = \beta_0 + \beta_1x_{i1} + \dots + \beta_px_{ip} + \epsilon_i,
\end{align}

donde, para el *i-ésimo* caso, $y_i$ es la variable de respuesta, $x_{ij}$ es la medición de la *j-ésima* variable, $b_j$ es el coeficiente de ésta variable y $\epsilon_i$ es el *error* (que queremos que sea lo más pequeño posible). 

Es decir, **la variable de respuesta es una *combinación lineal* de los regresores más un error**. Equivalentemente, se encuentra el hiperplano que 
"mejor se ajuste" a los datos cuando se minimiza el error.

## Algoritmo/Arquitectura

Crearemos un [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para facilitar el uso del modelo para las predicciones. 

El primer paso es un tranformador de columnas que se va a encargar de crear las variables dummies de las variables categóricas.

El segundo paso, la regresión lineal.

In [ ]:
transformer = ColumnTransformer(
    [
     ("dummies", OneHotEncoder(drop = 'first'), ['sex', 'smoker', 'region'])
    ], 
    remainder = 'passthrough'
)

lr_pipe = Pipeline(
    [
     ("transformador", transformer), 
     ('linear_regression', LinearRegression())
    ]
)

## Entrenamiento

Usamos la función ```fit``` para ajustar el modelo a nuestros datos.

In [ ]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('transformador',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummies',
                                                  OneHotEncoder(drop='first'),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('linear_regression', LinearRegression())])

## Evaluación

Con el modelo entrenado, usamos la función ```predict``` para obtener las predicciones del modelo con los datos que le suministramos.

Obtenemos las predicciones de ambos conjuntos de datos (entrenamiento y prueba).

In [ ]:
y_pred_train_lr = lr_pipe.predict(X_train)
y_pred_test_lr = lr_pipe.predict(X_test)

In [ ]:
verdaderos = y_test.reset_index(drop=True)
predichos = pd.Series(y_pred_test_lr, name='pred_charges').reset_index(drop=True)
comparativo = pd.concat([predichos, verdaderos], axis=1) 

In [ ]:
comparativo

,pred_charges,charges
0,8662.892824,7281.50560
1,6261.494309,5267.81815
2,15331.327092,12347.17200
3,11299.024976,24513.09126
4,4104.169420,3736.46470
...,...,...
397,34591.935065,24106.91255
398,8013.547530,17878.90068
399,32499.744617,22462.04375
400,2917.938751,1391.52870


### Métricas

Las métricas que usaremos son 3:

1. **Coeficiente de determinación ($R^2$)**: El mejor posible valor es 1 (toda la variabilidad de la variable dependiente se puede explicar con las variables independientes). Un modelo que siempre predice la media de la variable de respuesta tiene un resultado de 0. Modelos con peor desempeño que el que sólo predice la media, obtienen valores negativos.

2. **Error cuadrático promedio (MSE)**: El promedio de los errores al cuadrado.

3. **Error absoluto promedio (MAE)**: El promedio de los valores absolutos de los errores.

In [ ]:
print(
    f"R2 train: {r2_score(y_train, y_pred_train_lr)}", 
    f"R2 test: {r2_score(y_test, y_pred_test_lr)}", 
    "",
    f"MSE train: {mean_squared_error(y_train, y_pred_train_lr)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_lr)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_lr)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_lr)}", 
    sep = '\n'
)

R2 train: 0.7618780003618578
R2 test: 0.7166124432331722

MSE train: 36576361.89065711
MSE test: 36800107.88801562

MAE train: 4175.618401731289
MAE test: 4226.647664219914


El coeficiente de determinación de .76 en entramiento y .71 en prueba nos indica que el modelo se comporta de manera similar con datos vistos durante el entramiento y fuera de estos (aparentemente no hay sobreajuste).

### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_lr, y_test, y_pred_test_lr, "Regresión lineal")

En la gráfica se observan 3 grupos distintos de puntos, esto nos indica dos cosas: 

1. Puede que alguno de los supuestos del algoritmo no se cumplan.
2. Existen 3 o 4 grupos en los datos. Posiblemente aginando una nueva variable por grupos pueda mejorar las cosas.

# Primer Red Neuronal

La arquitectura de la primera red neuronal será muy similar a una regresión lineal, la única diferencia radica en el algoritmo de optimización de la función de pérdida.

## Preparación de los datos

Ocuparemos el mismo transformador que se creo y ajustó cuando usamos la regresión lineal. Los datos *preparados* nos van a servir para ambas redes neuronales.

In [ ]:
#X_train_transformed
X_train_tr = transformer.transform(X_train)
X_test_tr = transformer.transform(X_test)

In [ ]:
X_test

,age,sex,bmi,children,smoker,region
7,37,female,27.740,3,no,northwest
999,36,female,26.885,0,no,northwest
1209,59,male,37.100,1,no,southwest
491,61,female,25.080,0,no,southeast
625,29,female,26.030,0,no,northwest
...,...,...,...,...,...,...
854,49,female,23.845,3,yes,northeast
554,25,female,41.325,0,no,northeast
1278,39,male,29.925,1,yes,northeast
374,20,male,33.330,0,no,southeast


In [ ]:
X_test_tr

array([[ 0.   ,  0.   ,  1.   , ..., 37.   , 27.74 ,  3.   ],
       [ 0.   ,  0.   ,  1.   , ..., 36.   , 26.885,  0.   ],
       [ 1.   ,  0.   ,  0.   , ..., 59.   , 37.1  ,  1.   ],
       ...,
       [ 1.   ,  1.   ,  0.   , ..., 39.   , 29.925,  1.   ],
       [ 1.   ,  0.   ,  0.   , ..., 20.   , 33.33 ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ..., 46.   , 33.44 ,  1.   ]])

## Algoritmo/arquitectura

La arquitectura sólo consiste en una capa de entrada y una de salida con activación lineal. 

Al inicio ponemos un par de semillas para asegurarnos que los resultados se puedan replicar.

In [ ]:
X_train_tr.shape[1]

8

In [ ]:
np.random.seed(10)
tf.random.set_seed(10)

model = keras.Sequential(
    [
     layers.Input([X_train_tr.shape[1]]),
     layers.Dense(1, activation='linear')
    ]
)

Compilamos el modelo indicando que el optimizador es RMSprop, la función de pérdida el error cuadrático medio e indicamos que monitoreé el error absoluto medio.

In [ ]:
model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=.1), 
    loss = 'mse', 
    metrics = ['mae'], 
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


## Parámetros

Algunos ejemplos de otros parametros que podemos utilizar son:

- Optimizadores:
  - SGD
  - RMSprop
  - Adam
  - Adadelta
  - Adagrad
  - Adamax
  - Nadam
  - Ftrl

Más detalles [aquí](https://keras.io/api/optimizers/)

- Funciones de pérdida:
 - Binary Crossentropy
 - Categorical Crossentropy
 - Poisson
 - Mean Squared Error
 - Mean Absolute Error
 - Cosine Similarity

- Métricas:
  - Accuracy
  - Binary Crossentropy
  - AUC
  - MAE
  - MSE


Definimos el número de épocas, la ruta donde se deben guardar los checkpoints y creamos el callback que se encargará de guardar los checkpoints.

In [ ]:
epochs = 200

# weights_filepath = os.path.join('/content/drive/MyDrive/checkpoints/', 'model_1')

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#    filepath = weights_filepath , 
#    save_best_only = True, 
#    save_weights_only = True, 
#    monitor = 'val_mae', 
#    mode = 'min'
#)

El callback para los checkpoints sólo guardará los pesos (```save_weights_only = True```) del mejor modelo (```save_best_only = True```) considerando el error absoluto medio del conjunto de validación (```monitor = 'val_mae'```) más pequeño (```mode = 'min'```) hasta el momento.

## Entrenamiento

Ya que esta definida la arquitectura del modelo, se compilo y se crearon los callbacks necesarios, usamos la función ```fit``` para entrenar la red neuronal.

In [ ]:
history = model.fit(
    X_train_tr, y_train, 
    epochs=epochs, 
    validation_split=.2, 
#    callbacks=[model_checkpoint_callback]
    )

Epoch 1/200
24/24 [==============================] - 3s 9ms/step - loss: 337746464.0000 - mae: 13571.1084 - val_loss: 298195904.0000 - val_mae: 12156.6143
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 332252864.0000 - mae: 13382.3047 - val_loss: 293513088.0000 - val_mae: 11980.5703
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 327301600.0000 - mae: 13207.8291 - val_loss: 288939232.0000 - val_mae: 11806.2158
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 322428896.0000 - mae: 13035.0986 - val_loss: 284512512.0000 - val_mae: 11635.0654
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 317614176.0000 - mae: 12863.3105 - val_loss: 280186688.0000 - val_mae: 11465.4971
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 312897440.0000 - mae: 12691.9814 - val_loss: 275836704.0000 - val_mae: 11292.5635
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - los

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['mae'], name = 'entrenamiento')
)

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['val_mae'], name = 'validación')
)

fig.update_layout(
    hovermode = 'x unified', 
    legend_title = 'Conjunto de', 
    xaxis_title = 'Época', 
    yaxis_title = 'Error Absoluto Medio (MAE)', 
    title = 'MAE en cada época del entrenamiento<br><sup>Primera red neuronal</sup>', 
)


La gráfica muestra que el MAE llegó a un mínimo alrededor de la época 40 y luego el desempeño empeoró. Gracias a que agregamos el callback para guardar únicamente el mejor modelo basado en esta métrica, basta que lo carguemos con la función ```load_weights```.

In [ ]:
# model.load_weights(weights_filepath)

## Evaluación

Usamos la función ```predict``` para predecir usando el modelo que acabamos de entrenar.

In [ ]:
y_pred_train_nn1 = model.predict(X_train_tr)[:, 0]
y_pred_test_nn1 = model.predict(X_test_tr)[:, 0]

In [ ]:
y_pred_train_nn1

### Métricas

In [ ]:
print(
    f"MSE train: {mean_squared_error(y_train, y_pred_train_nn1)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_nn1)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_nn1)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_nn1)}", 
    sep = '\n'
)

MSE train: 131487903.45923795
MSE test: 117553262.1523611

MAE train: 9349.837116070214
MAE test: 8802.052779360774


### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_nn1, y_test, y_pred_test_nn1, 'Primera red neuronal')

Visualmente, se confirma lo que la $R^2$ nos indicó, se observan tres grupos de datos, y la red sólo aprendio a "predecir correctamente" uno de ellos.

Crearemos otra red con más capas esperando que se comporte mejor.

# Segunda Red Neuronal

## Algoritmo/Aquitectura

Para esta segunda red neuronal, estamos agregando 3 capas densas con 64 neuronas cada una, es decir, el modelo va a tener 5 capas en total:

1. Una capa de entrada,
2. tres capas ocultas, y
3. una capa de salida.

In [ ]:
np.random.seed(10)
tf.random.set_seed(10)

model = keras.Sequential(
    [
     layers.Input([X_train_tr.shape[1]]),
     layers.Dense(64, activation='relu'),
     layers.Dense(64, activation='relu'),
     layers.Dense(64, activation='relu'),     
     layers.Dense(1)
    ]
)

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(.01), 
    loss = 'mse', 
    metrics = ['mae']
)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                576       
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,961
Trainable params: 8,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 200

# weights_filepath = os.path.join('/content/drive/MyDrive/checkpoints/', 'model_2')

#model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#    filepath = weights_filepath, 
#    save_best_only = True, 
#    save_weights_only = True, 
#    monitor = 'val_mae', 
#    mode = 'min'
#)


## Entrenamiento

In [ ]:
history = model.fit(
    X_train_tr, y_train, 
    epochs=epochs, 
    validation_split=.2, 
#    callbacks=[model_checkpoint_callback]
    )

Epoch 1/200
24/24 [==============================] - 1s 9ms/step - loss: 290971008.0000 - mae: 11875.2041 - val_loss: 127459592.0000 - val_mae: 8538.9385
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 143272096.0000 - mae: 9402.0283 - val_loss: 126440896.0000 - val_mae: 8402.3633
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 135644624.0000 - mae: 9458.7236 - val_loss: 124784616.0000 - val_mae: 8468.6592
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 132211128.0000 - mae: 9474.9141 - val_loss: 123875232.0000 - val_mae: 8269.6895
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 130426632.0000 - mae: 9338.9160 - val_loss: 121591816.0000 - val_mae: 8537.9717
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 128815632.0000 - mae: 9182.7051 - val_loss: 119524736.0000 - val_mae: 8380.2158
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 12796412

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['mae'], name = 'entrenamiento')
)

fig.add_trace(
    go.Scatter(x = list(range(epochs)), y = history.history['val_mae'], name = 'validación')
)

fig.update_layout(
    hovermode = 'x unified', 
    legend_title = 'Conjunto de', 
    xaxis_title = 'Época', 
    yaxis_title = 'Error Absoluto Medio (MAE)', 
    title = 'MAE en cada época del entrenamiento<br><sup>Segunda red neuronal</sup>', 
)


Observamos un comportamiento "errático" en los valores del MAE a lo largo de cada época. Lo importante es que la gráfica muestra que el modelo sí aprendió (el valor de la métrica fue disminuyendo conforme avanzan las épocas de entrenamiento).

In [ ]:
# model.load_weights(weights_filepath)

## Evaluación

In [ ]:
y_pred_train_nn2 = model.predict(X_train_tr)[:, 0]
y_pred_test_nn2 = model.predict(X_test_tr)[:, 0]

### Métricas

In [ ]:
print(
    f"MSE train: {mean_squared_error(y_train, y_pred_train_nn2)}", 
    f"MSE test: {mean_squared_error(y_test, y_pred_test_nn2)}", 
    "",
    f"MAE train: {mean_absolute_error(y_train, y_pred_train_nn2)}", 
    f"MAE test: {mean_absolute_error(y_test, y_pred_test_nn2)}", 
    sep = '\n'
)

MSE train: 23372883.4040356
MSE test: 23407418.93006112

MAE train: 2878.041126533704
MAE test: 2947.652786269376


Obtenemos una $R^2$ mejor que cuando ajustamos el modelo con la regresión lineal y el MAE se redujo casi a la mitad. Parece que esta red aprendió mejor a predecir los datos.

### Visualización

In [ ]:
plot_model_evaluation(y_train, y_pred_train_nn2, y_test, y_pred_test_nn2, 'Segunda red neuronal')

Nuevamente, parece que hay 3 o 4 grupos de casos. Sin embargo, la diferencia es menos obvia que cuando se graficaron los resultados de la regresión lineal (y más aún en comparación a la primera red). 

# Conclusión

La regresión lineal fue un buen primer acercamiento a este problema considerando que arrojó una $R^2$ mayor a 0.7. Sin embargo, para cargos grandes no se comportó bien y se vieron 3 o 4 grupos distintos. Un análisis a detalle de las variables e ingeniería de características seguramente ayuda a mejorar el desempeño. 

El segundo intento con la red neuronal predice mejor que la anterior, es decir, logró *aprender* mejor las relaciones entre las variables. Con más capas, el desempeño mejora, pero se dejará como ejercicio.

# Opcionales

1. ¿Por qué se obtuvieron mejores resultados con la red neuronal que con la regresión lineal?

    1.1 ¿Por qué la primera red neuronal predice peor que la regresión lineal?
    
    1.2 ¿Cuál es la intuición detrás de "si hay más capas, usualmente la red neuronal predice mejor"?

2. Aplicar transformaciones a las variables y ajustar de nuevo la regresión lineal y una red neuronal para ver si mejoran los resultados.

3. Diseñar una red más compleja que mejore los resultados obtenidos

# Distintos modos de crear una red

In [ ]:
from tensorflow import keras

Creando el modelo capa por capa

In [ ]:
model = keras.Sequential(
    [
     keras.layers.Input([X_train_tr.shape[1]]),
     keras.layers.Dense(64, activation='relu'),
     keras.layers.Dense(64, activation='relu'),
     keras.layers.Dense(64, activation='relu'),     
     keras.layers.Dense(1)
    ]
)

Dando uno a uno los elementos y diciendo el orden que habrá

In [ ]:
data_input = keras.Input([X_train_tr.shape[1]])

dense_1 = keras.layers.Dense(64, activation='relu')(data_input)
dense_2 = keras.layers.Dense(64, activation='relu')(dense_1)
dense_3 = keras.layers.Dense(64, activation='relu')(dense_2)
output_layer = keras.layers.Dense(1)(dense_3)

model = keras.Model(inputs=[data_input], outputs=[output_layer])

Agregando capas al modelo

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1))

# Ligas de interés

- [Documentación de keras](https://keras.io/api/)
- [Documentación de Capas](https://keras.io/api/layers/)
- Mas documentación sobre optimizadores, métricas, funciones de perdida, etc. puede encontrarse en la documentación